In [30]:
##Imports
from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy import create_engine

In [31]:
##Store CSV into DataFrame

csv_file = "../Data/Netflix_titles.csv"
netflix_df = pd.read_csv(csv_file)
netflix_df.sample(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
3068,s4413,Movie,Naruto the Movie: Ninja Clash in the Land of Snow,Tensai Okamura,"Yuri Lowenthal, Maile Flanagan, Sam Regal, Cam...",Japan,1-Sep-17,2004,TV-PG,83 min,"Action & Adventure, Anime Features, Internatio...","Naruto, Sasuke and Sakura learn they'll be pro..."
7109,s5725,TV Show,Sotus The Series,NaN,"Sangpotirat Perawat, Ruangroj Prachaya",Thailand,25-May-19,2016,TV-MA,1 Season,"International TV Shows, Romantic TV Shows, TV ...",When freshman engineering student Kongpob stan...
5324,s7707,Movie,Yes or No,Saratswadee Wongsomphet,"Sushar Manaying, Supanart Jittaleela, Arisara ...",Thailand,6-Dec-18,2010,TV-PG,111 min,"International Movies, LGBTQ Movies, Romantic M...",Traditional girl Pie is initially uncomfortabl...
6815,s4732,TV Show,Pacific Heat,NaN,"Rob Sitch, Santo Cilauro, Tom Gleisner, Rebecc...",Australia,2-Dec-16,2016,TV-14,1 Season,"Crime TV Shows, TV Comedies","In this animated satire, a covert squad of ope..."
2430,s3459,Movie,Krystal,William H. Macy,"Rosario Dawson, Nick Robinson, Jacob Latimore,...",United States,1-Jan-19,2017,R,93 min,"Comedies, Dramas",A kid with a heart condition falls for an ex-s...


In [32]:
##Read Columns

for col in netflix_df.columns: 
    print(col)
    

show_id
type
title
director
cast
country
date_added
release_year
rating
duration
listed_in
description


In [33]:
##Drop all non-Movies and only capture U.S, and movies 2000-2017
netflix_df = netflix_df[(netflix_df.type == "Movie") & (netflix_df.release_year > 1999) & (netflix_df.release_year < 2018)] 
netflix_df.sample(10)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
4450,s6412,Movie,The Gospel of Matthew,David Batty,"Selva Rasalingam, Mourad Zaoui, Karima Gouit, ...",NaN,19-Oct-18,2014,TV-14,190 min,"Dramas, Faith & Spirituality",The Apostle Matthew is highlighted in this wor...
4727,s6821,Movie,The Rift: The Dark Side of the Moon,Dejan Zečević,"Ken Foree, Katarina Čas, Dragan Mićanović, Mon...","Serbia, South Korea, Slovenia",26-Feb-18,2016,TV-MA,92 min,"Horror Movies, Independent Movies, Sci-Fi & Fa...",When a team of special agents is sent to recov...
2477,s3531,Movie,Larceny,R. Ellis Frazier,"Dolph Lundgren, Corbin Bernsen, Louis Mandylor...","United States, Mexico",15-Sep-17,2017,TV-14,87 min,Action & Adventure,An ex-CIA contractor turned thief is forced to...
2604,s3724,Movie,Los Niños Héroes de Chapultepec,Ignacio López Escrivá,NaN,NaN,15-Dec-17,2017,TV-14,88 min,"Documentaries, International Movies",This documentary pays tribute to the young cad...
4442,s6397,Movie,The Girl with the Dragon Tattoo,David Fincher,"Daniel Craig, Rooney Mara, Christopher Plummer...","United States, Sweden, Norway",5-Jan-21,2011,R,158 min,"Dramas, Thrillers",When a young computer hacker is tasked with in...
808,s1097,Movie,Brian Regan: Nunchucks and Flamethrowers,Troy Miller,Brian Regan,United States,21-Nov-17,2017,TV-PG,61 min,Stand-Up Comedy,Brian Regan takes relatable family humor to ne...
4889,s7052,Movie,This Was Tomorrow,Wim Bonte,NaN,Belgium,15-Dec-15,2015,NR,77 min,"Documentaries, International Movies, Music & M...",The Tomorrowland music festival marks its 10th...
380,s489,Movie,An American Tail: The Treasures of Manhattan I...,Larry Latham,"Thomas Dekker, Dom DeLuise, Pat Musick, Nehemi...",United States,1-Apr-18,2000,G,79 min,Children & Family Movies,When Fievel and friends go hunting for buried ...
1729,s2472,Movie,Golmaal: Fun Unlimited,Rohit Shetty,"Ajay Devgn, Tusshar Kapoor, Arshad Warsi, Shar...",India,31-Dec-19,2006,TV-14,148 min,"Comedies, International Movies",Between thwarting crooks and wooing the belle ...
4324,s6218,Movie,The Christmas Candle,John Stephenson,"Hans Matheson, Samantha Barks, Lesley Manville...","United Kingdom, United States",1-Nov-19,2013,PG,100 min,"Dramas, Faith & Spirituality",A progressive new minister's arrival in a smal...


In [34]:
##Choose Columns, rename 

clean_netflix_df = netflix_df[['title', 'director', 'cast', 'release_year', 'rating', 'duration', 'listed_in', 'description']].copy()
clean_netflix_df = clean_netflix_df.rename(columns={'title': 'movie_title', 'cast': 'actors'})
clean_netflix_df = clean_netflix_df.drop_duplicates(subset='movie_title', keep='first')

clean_netflix_df=clean_netflix_df.drop_duplicates(['movie_title'] , keep ='first') 

clean_netflix_df = clean_netflix_df.reset_index(drop=True)

clean_netflix_df.to_csv("../Output/Netflix_clean.csv")

clean_netflix_df.head(5)

,movie_title,director,actors,release_year,rating,duration,listed_in,description
0,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
1,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
2,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
3,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...
4,1920,Vikram Bhatt,"Rajneesh Duggal, Adah Sharma, Indraneil Sengup...",2008,TV-MA,143 min,"Horror Movies, International Movies, Thrillers",An architect and his wife move into a castle t...


In [35]:
#Connect to Local Database

rds_connection_string = "postgres:postgres@localhost:5432/ETL_project_movie_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [37]:
#Check for tables
engine.table_names()

['imbd_data', 'netflix_data']

In [38]:
### Use pandas to load csv converted DataFrame into database
clean_netflix_df.to_sql(name='netflix_data', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "netflix_data_pkey"
DETAIL:  Key (movie_title)=(7:19) already exists.

[SQL: INSERT INTO netflix_data (movie_title, director, actors, release_year, rating, duration, listed_in, description) VALUES (%(movie_title)s, %(director)s, %(actors)s, %(release_year)s, %(rating)s, %(duration)s, %(listed_in)s, %(description)s)]
[parameters: ({'movie_title': '7:19', 'director': 'Jorge Michel Grau', 'actors': 'Demián Bichir, Héctor Bonilla, Oscar Serrano, Azalia Ortiz, Octavio Michel, Carmen Beato', 'release_year': 2016, 'rating': 'TV-MA', 'duration': '93 min', 'listed_in': 'Dramas, International Movies', 'description': 'After a devastating earthquake hits Mexico City, trapped survivors from all walks of life wait to be rescued while trying desperately to stay alive.'}, {'movie_title': '23:59', 'director': 'Gilbert Chan', 'actors': 'Tedd Chan, Stella Chung, Henley Hii, Lawrence Koh, Tommy Kuan, Josh Lai,

In [39]:
##Confirm data has been added by querying the netflix_data table
pd.read_sql_query('select * from netflix_data', con=engine).head()

,movie_title,year,director,actors,release_year,rating,duration,listed_in,description
0,7:19,None,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
1,23:59,None,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
2,9,None,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
3,21,None,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...
4,1920,None,Vikram Bhatt,"Rajneesh Duggal, Adah Sharma, Indraneil Sengup...",2008,TV-MA,143 min,"Horror Movies, International Movies, Thrillers",An architect and his wife move into a castle t...


In [ ]:
#See how many movies by release year

netflix_by_year = clean_netflix_df["release_year"].value_counts()
netflix_by_year.head()

In [20]:
#See how many movies by Rating

netflix_by_year = clean_netflix_df["rating"].value_counts()
netflix_by_year.head()

TV-MA    997
TV-14    779
R        462
TV-PG    298
PG-13    275
Name: rating, dtype: int64